## knock 076 カテゴリの割合に応じたサンプリングを行う(層化抽出)

In [14]:
import polars as pl
pl.Config.set_tbl_cols(-1)# 列の表示が省略されないようにする
import polars.selectors as cs# 抽出条件のプリセット

### データを読み込む

In [15]:
df_customer = pl.read_csv("../docker/work/data/customer.csv")
display(df_customer.head())

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,i64,str,str,i64,str,str,str,i64,str
"""CS021313000114…","""大野 あや子""",1,"""女性""","""1981-04-29""",37,"""259-1113""","""神奈川県伊勢原市粟窪****…","""S14021""",20150905,"""0-00000000-0"""
"""CS037613000071…","""六角 雅彦""",9,"""不明""","""1952-04-01""",66,"""136-0076""","""東京都江東区南砂******…","""S13037""",20150414,"""0-00000000-0"""
"""CS031415000172…","""宇多田 貴美子""",1,"""女性""","""1976-10-04""",42,"""151-0053""","""東京都渋谷区代々木*****…","""S13031""",20150529,"""D-20100325-C"""
"""CS028811000001…","""堀井 かおり""",1,"""女性""","""1933-03-27""",86,"""245-0016""","""神奈川県横浜市泉区和泉町**…","""S14028""",20160115,"""0-00000000-0"""
"""CS001215000145…","""田崎 美紀""",1,"""女性""","""1995-03-29""",24,"""144-0055""","""東京都大田区仲六郷*****…","""S13001""",20170605,"""6-20090929-2"""


### ノック

In [16]:
(
    df_customer
    # 使用する列を抽出する(大規模データの場合メモリ使用量を節約する効果がある)
    .select([
        pl.col("customer_id"),#1列だけだとSeriesになってしまうので、もう1列抽出する
        pl.col("gender_cd")
    ])
    # 性別コードで層別する
    .group_by(by = "gender_cd")
    # 性別コード毎に10%のサンプルを抽出する
    .map_groups(lambda group: group.sample(fraction = 0.1, seed = 42))
    # 性別コード毎のデータ数を計算する
    .select(
        pl.col('gender_cd').value_counts()
    )
    # 通常のDataFrameに戻す
    .unnest("gender_cd")
    # 性別コードの昇順に並べ替える
    .sort("gender_cd")
)

gender_cd,counts
i64,u32
0,298
1,1791
9,107
